# Batch Forecasting - Январь-Сентябрь 2025

Этот ноутбук выполняет прогнозирование по алгоритмам BASE и BASE+ для периода январь-сентябрь 2025 года.

## Настройка путей к файлам

In [ ]:
# ========================================
# ПУТИ К ФАЙЛАМ - НАСТРОЙТЕ ПЕРЕД ЗАПУСКОМ
# ========================================

# Путь к файлу с предыдущими прогнозами BASE
BASE_PREDICTIONS_FILE = r"d:\work\nwc\previous_predictions_BASE.xlsx"

# Путь к файлу с предыдущими прогнозами BASE+
BASE_PLUS_PREDICTIONS_FILE = r"d:\work\nwc\previous_predictions_BASE+.xlsx"

# Путь к файлу с историческими данными (ЧОК)
HISTORICAL_DATA_FILE = r"d:\work\nwc\DUMMY DATA.xlsx"

## Импорт библиотек и настройка

In [ ]:
import os
import re
import json
import scipy
import shutil
import numpy as np
import pandas as pd
import yaml
from typing import Dict, Any
from datetime import datetime, timedelta
from functools import reduce
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import MonthEnd

from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from autogluon.tabular import TabularPredictor

from utils.datahandler import *
from utils.config import *
from utils.predictors import *
from utils.common import *
from utils.pipelines import *

print("Библиотеки импортированы успешно")

## Загрузка конфигурации

In [ ]:
# Загрузка конфигурации
config = load_config('../config_refined.yaml')

DATE_COLUMN = config['DATE_COLUMN']
RATE_COLUMN = config['RATE_COLUMN']
ITEM_ID = config['item_id']
FACTOR = config['factor']
models_to_use = config['models_to_use']
TABPFNMIX_model = config['TABPFNMIX_model']
METRIC = config['metric_for_training']

# Статьи для прогноза
ITEMS_TO_PREDICT = config['Статья']

print(f"Конфигурация загружена")
print(f"Всего статей для прогноза: {len(ITEMS_TO_PREDICT)}")
print(f"Метрика: {METRIC}")

## Загрузка исторических данных

In [ ]:
# Загрузка и трансформация данных
print(f"Загрузка данных из {HISTORICAL_DATA_FILE}")
df_all_items = load_and_transform_data(HISTORICAL_DATA_FILE, DATE_COLUMN, RATE_COLUMN)

print(f"Данные загружены успешно")
print(f"Форма данных: {df_all_items.shape}")
print(f"Период данных: с {df_all_items[DATE_COLUMN].min()} по {df_all_items[DATE_COLUMN].max()}")
df_all_items.head()

## Определение месяцев для прогноза

In [ ]:
# Генерация списка месяцев для прогноза: январь - сентябрь 2025
months_to_forecast = [
    datetime(2025, 1, 1),
    datetime(2025, 2, 1),
    datetime(2025, 3, 1),
    datetime(2025, 4, 1),
    datetime(2025, 5, 1),
    datetime(2025, 6, 1),
    datetime(2025, 7, 1),
    datetime(2025, 8, 1),
    datetime(2025, 9, 1)
]

print(f"Месяцы для прогноза ({len(months_to_forecast)}):")
for month in months_to_forecast:
    print(f"  - {month.strftime('%B %Y')}")

## Запуск прогнозирования в цикле

In [ ]:
# Создаем папку results для бэкапов, если её нет
os.makedirs('../results', exist_ok=True)

# Инициализация файлов с прогнозами для каждого алгоритма
prev_base_file = BASE_PREDICTIONS_FILE
prev_base_plus_file = BASE_PLUS_PREDICTIONS_FILE

# Запуск прогнозирования для каждого месяца
total_time_start = datetime.now()

for idx, CHOSEN_MONTH in enumerate(months_to_forecast, 1):
    print("\n" + "="*80)
    print(f"ПРОГНОЗ {idx}/{len(months_to_forecast)}: {CHOSEN_MONTH.strftime('%B %Y')}")
    print("="*80)
    
    print(f"Начало прогнозирования для {CHOSEN_MONTH.strftime('%Y-%m')}")
    
    # Генерация периода для предиктов
    MONTHES_TO_PREDICT = generate_monthly_period(CHOSEN_MONTH)

    # ========================================
    # 2. ПРОГНОЗ BASE+
    # ========================================
    try:
        print("\n" + "-"*80)
        print(f"  Шаг 2/2: Прогноз BASE+ для {CHOSEN_MONTH.strftime('%B %Y')}")
        print("-"*80)
        
        time_start_plus = datetime.now()
        print(f"Запуск BASE+ пайплайна для {CHOSEN_MONTH.strftime('%Y-%m')}")
        
        # Временный файл для сохранения результатов
        temp_base_plus_file = '../results/temp_predict_BASE+.xlsx'
        
        run_base_plus_pipeline(
            df_all_items=df_all_items,
            ITEMS_TO_PREDICT=ITEMS_TO_PREDICT,
            config=config,
            DATE_COLUMN=DATE_COLUMN,
            RATE_COLUMN=RATE_COLUMN,
            ITEM_ID=ITEM_ID,
            FACTOR=FACTOR,
            models_to_use=models_to_use,
            TABPFNMIX_model=TABPFNMIX_model,
            METRIC=METRIC,
            CHOSEN_MONTH=CHOSEN_MONTH,
            MONTHES_TO_PREDICT=MONTHES_TO_PREDICT,
            result_file_name=temp_base_plus_file,
            prev_predicts_file=prev_base_plus_file
        )
        
        # Копируем результат в исходный файл (перезаписываем)
        shutil.copy2(temp_base_plus_file, BASE_PLUS_PREDICTIONS_FILE)
        
        # Также сохраняем бэкап в results
        backup_base_plus_file = '../results/predict_BASE+.xlsx'
        shutil.copy2(temp_base_plus_file, backup_base_plus_file)
        
        # Удаляем временный файл
        os.remove(temp_base_plus_file)
        
        time_finish_plus = datetime.now()
        runtime_plus = time_finish_plus - time_start_plus
        
        print(f"  ✓ BASE+ прогноз для {CHOSEN_MONTH.strftime('%Y-%m')} завершен за {runtime_plus}")
        print(f"  ✓ Результаты BASE+ сохранены в: {BASE_PLUS_PREDICTIONS_FILE}")
        print(f"  ✓ Бэкап сохранен в: {backup_base_plus_file}")
        
        # Обновляем путь к файлу BASE+ для следующей итерации
        prev_base_plus_file = BASE_PLUS_PREDICTIONS_FILE
        
    except Exception as e:
        print(f"  ✗ ОШИБКА при BASE+ прогнозировании для {CHOSEN_MONTH.strftime('%Y-%m')}: {str(e)}")
        print("Прогнозирование прервано")
        raise
    
    # Итоговое время для месяца
    total_month_time = time_finish_plus - time_start_plus
    print(f"\n  Общее время для {CHOSEN_MONTH.strftime('%B %Y')}: {total_month_time}")
    print(f"    - BASE+: {runtime_plus}")

total_time_finish = datetime.now()
total_runtime = total_time_finish - total_time_start

print("\n" + "="*80)
print("ПРОГНОЗИРОВАНИЕ ЗАВЕРШЕНО")
print("="*80)
print(f"Общее время выполнения: {total_runtime}")
print(f"Среднее время на месяц: {total_runtime / len(months_to_forecast)}")
print(f"\nИтоговые файлы с прогнозами:")
print(f"  - BASE+: {BASE_PLUS_PREDICTIONS_FILE}")
print(f"\nБэкапы сохранены в папке ../results/")
print(f"\nBatch прогнозирование завершено. Общее время: {total_runtime}")